In [1]:
import pandas as pd
import numpy as np
from dash import dcc, html, Input, Output
import dash
import plotly.graph_objs as go

# Create a function to generate sample data
def create_dataset():
    # Create a date range
    date_range = pd.date_range(start='1980-01-01', end='2022-12-31', freq='M')

    # Create a DataFrame
    data = {
        'Date': date_range,
        'Recession': np.where(
            ((date_range.year == 1980) |
             ((date_range.year >= 1981) & (date_range.year <= 1982)) |
             (date_range.year == 1991) |
             ((date_range.year >= 2000) & (date_range.year <= 2001)) |
             ((date_range.year >= 2007) & (date_range.year <= 2009)) |
             ((date_range.year == 2020) & (date_range.month.isin([2, 3, 4])))), 1, 0),
        'Automobile_Sales': np.random.randint(100, 1000, size=len(date_range)),
        'GDP': np.random.randint(20000, 60000, size=len(date_range)),
        'Unemployment_Rate': np.random.uniform(3, 10, size=len(date_range)),
        'Consumer_Confidence': np.random.randint(50, 100, size=len(date_range)),
        'Seasonality_Weight': np.random.uniform(0.5, 1.5, size=len(date_range)),
        'Price': np.random.randint(20000, 50000, size=len(date_range)),
        'Advertising_Expenditure': np.random.randint(5000, 20000, size=len(date_range)),
        'Vehicle_Type': np.random.choice(['Superminicar', 'Smallfamilycar', 'Mediumfamilycar', 'Executivecar', 'Sports'], size=len(date_range)),
    }

    df = pd.DataFrame(data)
    df['Month'] = df['Date'].dt.month
    return df

# Create the dataset
df = create_dataset()

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Automobile Sales Dashboard"),
    
    # Dropdown for Vehicle Type selection
    dcc.Dropdown(
        id='vehicle-type-dropdown',
        options=[{'label': vehicle, 'value': vehicle} for vehicle in df['Vehicle_Type'].unique()],
        value=df['Vehicle_Type'].unique()[0],  # Default value
        multi=False,
        clearable=False,
        className='dropdown',
        placeholder="Select a Vehicle Type"
    ),

    # Dropdown for Recession status selection
    dcc.Dropdown(
        id='recession-dropdown',
        options=[
            {'label': 'During Recession', 'value': '1'},
            {'label': 'Not During Recession', 'value': '0'}
        ],
        value='0',  # Default value
        multi=False,
        clearable=False,
        className='dropdown',
        placeholder="Select Recession Status"
    ),

    # Dropdown for statistics selection
    dcc.Dropdown(
        id='statistic-dropdown',
        options=[
            {'label': 'Total Sales', 'value': 'total'},
            {'label': 'Average Sales', 'value': 'average'}
        ],
        value='total',  # Default value
        multi=False,
        clearable=False,
        className='dropdown',
        placeholder="Select Statistic"
    ),
    
    # Division for output display
    html.Div(id='output-display', className='output-display', style={'margin-top': '20px'}),
    
    # Graph for sales statistics
    dcc.Graph(id='sales-graph')
])

# Callback to update output display and graph based on dropdown selections
@app.callback(
    [Output('output-display', 'children'),
     Output('sales-graph', 'figure')],
    [Input('vehicle-type-dropdown', 'value'),
     Input('recession-dropdown', 'value'),
     Input('statistic-dropdown', 'value')]
)
def update_output_and_graph(vehicle_type, recession_status, statistic):
    filtered_data = df[(df['Vehicle_Type'] == vehicle_type) & (df['Recession'].astype(str) == recession_status)]
    
    # Calculate result based on the selected statistic
    if statistic == 'total':
        result = filtered_data['Automobile_Sales'].sum()
        output_message = f'Total Automobile Sales for {vehicle_type} during {"Recession" if recession_status == "1" else "Non-Recession"}: {result}'
    elif statistic == 'average':
        result = filtered_data['Automobile_Sales'].mean()
        output_message = f'Average Automobile Sales for {vehicle_type} during {"Recession" if recession_status == "1" else "Non-Recession"}: {result:.2f}'
    
    # Create a bar chart for the selected vehicle type and recession status
    sales_data = filtered_data.groupby('Date')['Automobile_Sales'].sum().reset_index()
    
    figure = {
        'data': [
            go.Bar(
                x=sales_data['Date'],
                y=sales_data['Automobile_Sales'],
                name='Sales',
                marker=dict(color='blue')
            )
        ],
        'layout': go.Layout(
            title=f'Sales for {vehicle_type} during {"Recession" if recession_status == "1" else "Non-Recession"}',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Automobile Sales'},
            barmode='group'
        )
    }
    
    return output_message, figure

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8055)  # Change port if necessary


C:\Users\saik7\AppData\Local\Temp\ipykernel_36592\3894570670.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start='1980-01-01', end='2022-12-31', freq='M')
